# Exercise07 : 再トレーニングとモデル評価

この演習では、機械学習パイプラインのシンプルな評価テストとして、新しいモデルと既存のモデルを比較します。新しいモデルの精度が優れている場合にのみ、そのモデルを昇格させます。そうでない場合、そのモデルは Azure Machine Learning モデルレジストリに登録されないようにします。



## 1. モデル評価スクリプトをファイルとして保存する (evaluate.py)

この例では、モデルの学習、モデル登録のためのパイプラインを作成します。
このパイプラインでは、以下の手順が実行される。

1. モデルの学習が行われる。
2. モデルの精度を評価し、精度が既存のモデルを上回ればフラグを更新する。
3. フラグに基づいてモデルをモデルレジストリに登録する。

そして、各ソースコードは以下のように保存される。

- training script ```./scripts/train.py```
- evaluate script ```./scripts/evaluate.py```
- register script ```./scripts/register_pipeline.py```

モデル名（モデルディレクトリのサブフォルダ名）は、モデル情報ファイル（JSONテキスト）に保存され、次のステップに渡されます。

In [ ]:
%%writefile scripts/evaluate.py

import argparse
import json
import mlflow
from mlflow.pyfunc import load_model
from mlflow.tracking import MlflowClient

def parse_args():

    parser = argparse.ArgumentParser()
    parser.add_argument('--model_name', type=str, help='Name under which model will be registered')
    parser.add_argument('--model_path', type=str, help='Model directory')
    parser.add_argument('--output_path', type=str, help="eval output", default='./outputs')

    args, _ = parser.parse_known_args()
    print(f'Arguments: {args}')

    return args

def main():

    args = parse_args()

    model_name = args.model_name
    model_path = args.model_path
    output_path = args.output_path
    deploy_flag = 0
    
    mlflow.set_tag("model_name", model_name)
    mlflow.set_tag("model_path", model_path)

    # For information transfer between pipelines
    json_open = open(model_path + "/metric.json", 'r')
    json_load = json.load(json_open)
    # Get accuracy (RMSE) from previous step Run
    run_rmse = json_load["RMSE"]
    print("run_rmse: " + str(run_rmse))
    run_r2 = json_load["R2"]
    print("run_r2: " + str(run_r2))

    run_id = json_load["run_id"]
    print("train run_id: " + str(run_id))
    # Get Run executed before
    finished_mlflow_run = mlflow.get_run(run_id)
    # Pull metrics and tags from Run
    metrics = finished_mlflow_run.data.metrics
    print(metrics)

    output_info = {
        'run_rmse' : run_rmse,
        'model_rmse' : 0,
        'deploy_flag' : deploy_flag,
    }

    try:
        # Get the latest registered models
        client = MlflowClient() 
        model_info = client.get_registered_model(model_name)
        print("model_info: " + str(model_info.latest_versions[0]))
        model_tags = model_info.latest_versions[0].tags
        json_metrics = json.loads(model_tags["metrics"])

        # Model accuracy (RMSE)
        model_rmse = json_metrics["RMSE"]
        output_info['model_rmse'] = model_rmse
        print("model_rmse: " + str(model_rmse))
        
        # RMSE Comparison
        if run_rmse < model_rmse:
            print("Improved accuracy. 精度が上回りました")
            deploy_flag = 1

        else:
            print("Accuracy did not improve. 精度が上回りませんでした")
            deploy_flag = 0
    
    except:
        print("No model exists for comparison. Register the model as it is. 比較対象のモデルが存在しません。そのまま登録します。")
        deploy_flag = 1

    output_info['run_rmse'] = run_rmse
    output_info['run_r2'] = run_r2
    output_info['deploy_flag'] = deploy_flag
    mlflow.set_tag("deploy_flag", deploy_flag)

    output_json = json.dumps(output_info)
    with open(output_path + "/output_evaluate.json", "w") as f:
        f.write(output_json)

    mlflow.log_artifact(output_path)

if __name__ == "__main__":
    main()


## 2. パイプライン用モデル登録スクリプトをファイルとして保存する (register_pipeline.py)
パイプライン実行順を指定するためにデータ入力引数を追加したバージョンです。

In [ ]:
%%writefile scripts/register_pipeline.py

import argparse
import json
import mlflow
from mlflow.pyfunc import load_model
from mlflow.tracking import MlflowClient

def parse_args():

    parser = argparse.ArgumentParser()
    parser.add_argument('--model_name', type=str, help='Name under which model will be registered')
    parser.add_argument('--model_path', type=str, help='Model directory')
    parser.add_argument('--deploy_flag', type=str, help='A deploy flag whether to deploy or no')
    parser.add_argument('--eval_path', type=str, help='eval directory')
    
    args, _ = parser.parse_known_args()
    print(f'Arguments: {args}')

    return args


def main():

    args = parse_args()

    model_name = args.model_name
    model_path = args.model_path
    eval_path = args.eval_path

    mlflow.set_tag("model_name", model_name)
    mlflow.set_tag("model_path", model_path)

    # For information transfer between pipelines
    json_open = open(eval_path + "/output_evaluate.json", 'r')
    json_load = json.load(json_open)
    run_rmse = json_load["run_rmse"]
    run_r2 = json_load["run_r2"]
    deploy_flag = json_load["deploy_flag"]
    print("run_rmse: " + str(run_rmse))
    print("run_r2: " + str(run_r2))
    print("deploy_flag: " + str(deploy_flag))

    mlflow.set_tag("run_rmse", run_rmse)
    mlflow.set_tag("run_r2", run_r2)
    mlflow.set_tag("deploy_flag", deploy_flag)

    if deploy_flag==1:
        # Load model from model_path
        model = load_model(model_path + "/models") 
        # Log the sklearn model and register as version 1
        modelreg = mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path="models",
            registered_model_name=model_name
        )
        print(modelreg)
        
        #Since log_model cannot register tags in models, use set_model_version_tag instead.
        client = MlflowClient() 
        model_info = client.get_registered_model(model_name)
        model_version = model_info.latest_versions[0].version
        dict_metrics =  {"RMSE": run_rmse, "R2": run_r2}
        client.set_model_version_tag(model_name, str(model_version), "metrics", json.dumps(dict_metrics))
        print("Model registered!")
    else:
        print("Model will not be registered!")

if __name__ == "__main__":
    main()




## 3. トレーニングスクリプトをファイルとして保存する (train.py)

Exercise02 で作成したトレーニングスクリプト `/scripts/train.py` を再利用します。

## 4. MLパイプラインの構築と実行

yaml でパイプラインを構成し、生成されたパイプラインに対してジョブを投入してみましょう。

> 注意 : この例では、 ``diabetes_data_oh4ml`` という名前の登録済みデータ アセットをコンピュートターゲットにマウントして使用します。
Exercise04 よりもデータ量が増えており、精度が向上する想定のサンプルデータです。
データ アセット準備のため、"[Exercise00-2 : Prepare Data](./exercise00_1_prepare_data.ipynb)" を実行します。

In [ ]:
%%writefile 07_training_pipeline_job_3step.yml
$schema: https://azuremlschemas.azureedge.net/latest/pipelineJob.schema.json
type: pipeline
experiment_name: 07_pipeline_oh4ml_3step
description: 3-Step Pipeline Example (Train, Evaluate, Register)

# <inputs_and_outputs>
inputs:
  input: 
    type: uri_file
    path: azureml:diabetes_data_oh4ml@latest
    #path: azureml:diabetes_data_oh4ml_350records@latest
  model_name: diabetes_model_oh4ml
outputs: 
  outputs:

settings:
  default_datastore: azureml:workspaceblobstore
  default_compute: azureml:demo-cpucluster1
  continue_on_step_failure: false

jobs:
  train_model:
    name: train_model
    display_name: train-model
    code: ./scripts
    command: >-
      python train.py 
      --input_data ${{inputs.diabetes_data}} 
      --output_dir ${{outputs.model_output}}
    environment: azureml:diabetes-env@latest
    inputs:
      diabetes_data: ${{parent.inputs.input}}
    outputs:
      model_output: ${{parent.outputs.outputs}}

  evaluate_model:
    name: evaluate_model
    display_name: evaluate-model
    code: ./scripts
    command: >-
      python evaluate.py 
      --model_name ${{inputs.model_name}} 
      --model_path ${{inputs.model_path}} 
      --output_path ${{outputs.evaluate_output}}
    environment: azureml:diabetes-env@latest
    inputs:
      model_name: ${{parent.inputs.model_name}}
      model_path: ${{parent.jobs.train_model.outputs.model_output}}
    outputs:
      evaluate_output: 

  register_model:
    name: register_model
    display_name: register-model
    code: ./scripts
    command: >-
      python register_pipeline.py 
      --model_name ${{inputs.model_name}} 
      --model_path ${{inputs.model_path}} 
      --deploy_flag ${{inputs.deploy_flag}}
      --eval_path ${{inputs.eval_path}}
    environment: azureml:diabetes-env@latest
    inputs:
      model_name: ${{parent.inputs.model_name}}
      model_path: ${{parent.jobs.train_model.outputs.model_output}}
      deploy_flag: 1
      eval_path: ${{parent.jobs.evaluate_model.outputs.evaluate_output}}


このパイプラインを実行するためのジョブを投入します。

In [ ]:
!az ml job create --file 07_training_pipeline_job_3step.yml

AML studio UI にアクセスし、パイプラインの結果をジョブで確認します。(下記参照)

![Pipeline results](../images/006.png)

パイプラインの出力でモデルメトリクスを抽出することができます。<br>
このトレーニングパイプラインがパスされた場合、MLOps 統合の次のステージを呼び出すことができます。

## 5.  コンピュート クラスターの削除

AML コンピュート クラスターは非アクティブになると自動的にノードが終了するので、コスト節約のために削除する必要はありません。しかし、もしクリーンアップしたい場合は、以下のように実行してください。

In [ ]:
"""
!az ml compute delete --name demo-cpucluster1 \
  --yes
"""